Getting started

In [1]:

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings

ModuleNotFoundError: No module named 'textblob'

In [ ]:
data = pd.read_csv('twitter_training.csv',header=None)
v_data = pd.read_csv('twitter_validation.csv',header=None)

EDA

In [ ]:
data.columns = ['#', 'refers to', 'sentiment', 'text']
v_data.columns = ['#', 'refers to', 'sentiment', 'text']


In [ ]:
data = data.drop(['#', 'refers to' ], axis=1)
v_data = v_data.drop(['#', 'refers to' ], axis=1)

Adding colunm names

Finding Null Values

In [ ]:
data.dropna(inplace=True, axis=0)

remove duplicate

In [ ]:
data=data.drop_duplicates()

In [ ]:
v_data=v_data.drop_duplicates()

Preprocessing



lowercase
pentu remove 
https\S+|www\S+https\S+\\@w+|\# remove
stop word remove
stem



In [ ]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [ ]:
data.text = data['text'].apply(data_processing)

In [ ]:
v_data.text = v_data['text'].apply(data_processing)

In [ ]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [ ]:
data['text'] = data['text'].apply(lambda x: stemming(x))
v_data['text'] = v_data['text'].apply(lambda x: stemming(x))

In [ ]:
vect = CountVectorizer(ngram_range=(1,2)).fit(data['text'])

In [ ]:
feature_names = vect.get_feature_names()


In [ ]:
x_train =  vect.transform(data['text'])
y_train = data['sentiment']
x_test =  vect.transform(v_data['text'])
y_test = v_data['sentiment']


The CountVectorizer will select the words/features/terms which occur the most frequently

In [ ]:
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
hi = logreg.predict('I am sad')

In [ ]:
print(hi)

In [ ]:
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)

In [ ]:
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.2f}%".format(svc_acc*100))

In [ ]:
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))